# Goal: Develop Kalman filter and smoother using Cython and compare to numpy implementation 

In [8]:
import Cython
import cython
import scipy
import pandas as pd
import numpy as np

%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


- A contigious array is an array whose elements are stored in adjecant memory block. A C-contigious array is an array whose adjacent elements are in rows together, whereas F-contigious is the Columns, so opposite what you'd expect based on the first letters
- Can only ommit trailing arguments in cdef functions. If cpdef, need the gil because python arguments were returned. Simply changing the order of arguments fixed like 10 errors in one step... All the errors required the gil. 

In [2]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, dgetri 
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_vcopy(double[::1] x, double[::1] y, int N, int incX=1, int incY=1) nogil:
    """
    
    custom copy
        
        arguments
            n: vectors x and y, copies x into y
        
        returns: void
        
    """
    dcopy(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double cddot(int N, double[::1] x, int incX, double[::1] y, int incY) nogil:
    """
    
    custom dot product
        
        arguments
            n: length of the vectors x and y
        
        returns: type double
        
    """
    return ddot(&N, &x[0], &incX, &y[0], &incY)

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void cAx(double[::1, :] A, double[::1] x, double[::1] y, int incX=1, int incY=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef double c_xAx(double[::1, :] A, double[::1] x, double[::1] Ax, double[::1] xAx, int incX=1, int inc_Ax=1, double alpha=1.0, double beta=0.0) nogil:
    """
    
    custom matrix vector x'Ax
    
        output: xAx
        
        returns: type double (can do arithmetic in C)
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &Ax[0], &inc_Ax)
    return ddot(&M, &x[0], &incX, &Ax[0], &inc_Ax)


@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_mm(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    """
    
    Matrix matrix multiplication AB
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_mm_transpose(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3) nogil:
    """
    
    Matrix matrix_transpose multiplication AB'
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    cdef double alpha = 1.0
    cdef double beta = 0.0
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_kalman_iteration(double[::1, :] T, double[::1, :] Z, double[::1, :] R,
                              double[::1, :] Q, double[::1, :] H, double[::1] a, 
                              double[::1, :] P, double[::1] y, 
                              double[::1] v, double[::1] M, int p, int s) nogil:
    """
    
    Kalman iteration
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """

    ### compute v
    
    # copy y into v, this is where v will be stored 
    c_vcopy(y, v, s)
    
    # v -> v - Za = y + alpha * Za because of copy line above
    with gil:
        cAx(Z, a, v ,alpha=-1)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_kalman_filter(double[::1, :] T, double[::1, :] Z, double[::1, :] R,
                              double[::1, :] Q, double[::1, :] H, double[::1, :] a, 
                              double[::1, :, :] P, double[::1, :] y, 
                           double[::1, :] v, double[::1, :] M, int p, int s, int n):
    """
    
    Kalman filter
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    # testing, when assigning atemp to a, and adding 1 to a temp, will a be 1,2,3...?
    
    cdef double[::1] a_temp = np.ones((2,1), order='f')

    for i in range(n):
        
        # a[i,:], which is c contigious, is assigned a pointer to a_temp, which is fortran contigious
        a[i, :] = a_temp
        
        #c_kalman_iteration(T, Z, R, Q, H, a_temp, 
        #                      P[i, :, :], y[i, :], v[i, :], M[i, :], p, s) 
        

In [11]:
%%cython

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, dgetri
from libcpp cimport bool

@cython.boundscheck(False)
@cython.wraparound(False)
cdef void c_vcopy(double[::1] x, double[::1] y, int N, int incX=1, int incY=1) nogil:
    """
    
    custom copy
        
        arguments
            n: vectors x and y, copies x into y
        
        returns: void
        
    """
    
    dcopy(&N, &x[0], &incX, &y[0], &incY)


@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cAx(double[::1, :] A, double[::1] x, double[::1] y, double alpha=1.0, double beta=0.0, int incX=1, int incY=1) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    dgemv('N', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef void cATx(double[::1, :] A, double[::1] x, double[::1] y, double alpha=1.0, double beta=0.0, int incX=1, int incY=1) nogil:
    """
    
    custom matrix vector alpha * Ax + beta y
    
        output: stored in y
        
    """
    
    cdef int M = A.shape[0]
    cdef int N = A.shape[1]
    
    # how many bits of memory are between A[i, j] and A[i, j + 1] -> the number of rows since A is fortran contigious
    cdef int LDA = M
    
    dgemv('T', &M, &N, &alpha, &A[0, 0], &LDA, &x[0], &incX, &beta, &y[0], &incY)
    

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_MM(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix multiplication AB
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    cdef int N = M2.shape[1]
    
    with gil:
        print(f'First matrix shape: {np.asarray(M1).shape}')
        print(f'First matrix shape: {np.asarray(M2).shape}')
        print(f'Output matrix shape: {np.asarray(M3).shape}')
        print()
        if K != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)
    #with gil:
    #    print(np.asarray(M3))

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_MTM(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix multiplication AB
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
   
    # switch for the transposed version
    cdef int M = M1.shape[1]
    cdef int K = M1.shape[0]
    
    cdef int N = M2.shape[1]
    
    with gil:
        print(f'First matrix shape: {np.asarray(M1).T.shape}')
        print(f'Second matrix shape: {np.asarray(M2).shape}')
        print(f'Output matrix shape: {np.asarray(M3).shape}')
        print()
        if M != M2.shape[0]:
            raise ValueError('dimension mismatch')
    
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('T', 'N', &M, &N, &K, &alpha, &M1[0,0], &M,
                                             &M2[0,0], &K, &beta,
                                             &M3[0,0], &M,)
    #with gil:
    #    print(np.asarray(M3))

    
    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_MMT(double[::1, :] M1, double[::1, :] M2, double[::1, :] M3, double alpha = 1.0, double beta = 0.0) nogil:
    """
    
    Matrix matrix_transpose multiplication AB'
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """
    
    cdef int M = M1.shape[0]
    cdef int K = M1.shape[1]
    
    # try to use the shape of the transposed matrix
    cdef int N = M2.shape[0]
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgemm('N', 'T', &M, &N, &K, &alpha, &M1[0,0], &M,
                                        &M2[0,0], &N, &beta,
                                        &M3[0,0], &M,)
    #with gil:
    #    print(np.asarray(M3))

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_inv(double[::1, :] A) nogil:
    """
    
    Matrix inversion
        
        returns: void, but result stored in A
        
    """
   
    cdef int N = A.shape[0]
    
    with gil:
        if N != A.shape[1]:
            raise ValueError('Matrix should be square')
    
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgetri(&N, &A[0, 0], &N)

    
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef void c_kalman_iteration(double[::1, :] Z, double[::1] a, 
                              double[::1] y, double[::1] v,
                              double[::1, :] P, double[::1, :] H,
                              double[::1, :] F, double[::1, :] F_inv, 
                              double[::1, :] I,  double[::1, :] ZP) nogil:
    """
    
    Kalman iteration
    
        output: matrix product
        
        returns: void, but result stored in M3
        
    """

    #### compute v = y + Za
    
    # copy y into v, this is where v will be stored
    c_vcopy(y, v, 2)
    
    # v -> v - Za = y + alpha * Za because of copy line above
    cATx(Z, a, v , alpha= -1.0)
    
    #### compute F = ZPZ' + H
    
    # F_ = ZP + 0F = ZP
    c_MM(Z, P, ZP)
    
    # F. = F_Z + 0*F = ZPZ'
    c_MMT(Z, ZP, F)
    
    # F = F. + H = ZPZ' + H
    c_MM(I, H, F, alpha=1.0, beta = 1.0)
    
    # copy F into F_inv
    F_inv = c_MM(F, I, F_inv)
    
    # compute inverse of F
    
    # compute M and K


Error compiling Cython file:
------------------------------------------------------------
...

cimport numpy as np
import numpy as np
import cython
from scipy.linalg.cython_blas cimport dgemm, ddot, dgemv, daxpy, dcopy, dgetri
^
------------------------------------------------------------

C:\Users\Ota Niels\.ipython\cython\_cython_magic_d0809234edf7d5d2716c1b9d7d2c799d.pyx:5:0: 'scipy\linalg\cython_blas\dgetri.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...
    
    
    # Q: LDA, how many elements do you need to jump over to go from A[i,j] -> A[i, j+1] if A column major
    # A: Exactly the number of rows in A
    
    dgetri(&N, &A[0, 0], &N)
   ^
------------------------------------------------------------

C:\Users\Ota Niels\.ipython\cython\_cython_magic_d0809234edf7d5d2716c1b9d7d2c799d.pyx:192:4: 'dgetri' is not a constant, variable or function identifier

Error compiling Cython file:
---------------------------------

In [3]:
# system variables
a = np.zeros((2,), order='f') + 1
Z = np.zeros((1, 2), order='f') + 1
v = np.zeros((1,), order='f')
y = np.zeros((1,), order='f') + 1
P = np.eye(2, order='f')
H = np.zeros((1,1), order='f') + 1
F = np.zeros((1,1), order='f')

# helper variables
I = np.eye(1, order='f')
ZP = np.zeros((Z.shape[0], Z.shape[1]), order='f') 

In [4]:
print(f'a: {a} with shape: {a.shape}')
print(f'Z: {Z} with shape: {Z.shape}') 
print(f'v: {v} with shape: {v.shape}')
print(f'y: {y} with shape: {y.shape}')
print(f'P: {P} with shape: {P.shape}')
print(f'H: {H} with shape: {H.shape}') 
print(f'F: {F} with shape: {F.shape}')

print(f'ZP: {ZP} with shape: {ZP.shape}')

a: [1. 1.] with shape: (2,)
Z: [[1. 1.]] with shape: (1, 2)
v: [0.] with shape: (1,)
y: [1.] with shape: (1,)
P: [[1. 0.]
 [0. 1.]] with shape: (2, 2)
H: [[1.]] with shape: (1, 1)
F: [[0.]] with shape: (1, 1)
ZP: [[0. 0.]] with shape: (1, 2)


In [5]:
c_kalman_iteration(Z, a, y, v, P, H, F, I, ZP)

First matrix shape: (1, 2)
First matrix shape: (2, 2)
Output matrix shape: (1, 2)

First matrix shape: (1, 1)
First matrix shape: (1, 1)
Output matrix shape: (1, 1)



In [7]:
print(f'a: {a}')
print(f'Z: {Z}') 
print(f'v: {v}')
print(f'y: {y}')
print(f'P: {P}')
print(f'H: {H}') 
print(f'F: {F}')
print(f'ZP: {ZP}')
print(f'I: {I}')

a: [1. 1.]
Z: [[1. 1.]]
v: [-1.]
y: [1.]
P: [[1. 0.]
 [0. 1.]]
H: [[1.]]
F: [[3.]]
ZP: [[1. 1.]]
I: [[1.]]


In [7]:
A = np.zeros((1, 2), order='f') + 1
B = np.zeros((1, 2), order='f') + 1
AB  = np.zeros((1,1), order='f')

c_MMT(A, B, AB)

First matrix shape: (1, 2)
Second matrix shape: (2, 1)
Output matrix shape: (1, 1)



In [8]:
AB

array([[2.]])